# Yi-Hsuan Chen's Python module

**import yhc_module as yhc**

**available functions**
1. unit_conversion

In [1]:
import numpy as np
import xarray as xr
import io, os, sys, types

In [ ]:
#############################################
# Purpose of this code:
#
# 1. convert yhc_module.ipynb to yhc_module.py
# 2. copy yhc_module.py to a a folder, dir_py, so it can be imported
#############################################

convert_ipynb2py = False
#convert_ipynb2py = True

dir_py="/Users/yihsuan/.ipython"  # (Mac) copy yhc_module to this folder 
                                  #       so that other jupyter notebook can import yhc_module

# location of yhc_module in Mac
py_path = "/Users/yihsuan/Downloads/yihsuan/test/tool/python/yhc_module_and_notes"

if (convert_ipynb2py):
    print("convert yhc_module.ipynb to yhc_module.py")
    command="jupyter nbconvert yhc_module.ipynb --to python"
    os.system(command)
    
    command="cp yhc_module.py "+dir_py
    print(command)
    os.system(command)
    
    sys.path.append(py_path)  # add py_path to sys.path
    

In [ ]:
def printv(var,
           text = "", 
          ):
    """
    ----------------------
    Description:
      print out variables by begining with a line, follow by a text, and end by a empty line
      This helps to view the output more easily.

    Input arguments:
      var : any variable
      text: text will show on screen

    Return:
      print statement on screen

    Example:
      import yhc_module as yhc
      
      yhc.printv(var, "ggg")
      yhc.printv(var, text = "aaa")

    Date created: 2022/07/06
    ----------------------
    """    

    print("--------------")
    print(text)
    print(var)
    print("")

In [ ]:
def unit_conversion (var, unit_in, unit_out):
    """
    ----------------------
    convert unit of a variable
    
    Input arguments:
        var      : (xarray.DataArray): a variable
        unit_in  : (string) original units of the var
        units_out: (string) new units of the var 
    
    Return:
        variable values with unit_out
    
    Example
        import yhc_module as yhc
        var = yhc.unit_conversion(var, "m", "km")
        
    Date created: 2022/06/29
    -----------------------
    """
    
    # constants 
    rho_water = 1000.  # water density [kg/m3]
    latent_heat_evap = 2.5e+6          # latent heat of vaporization for water, J/kg  
    latent_heat_cond = 1./latent_heat_evap  # latent heat of vaporization for water, J/kg
    hr2sec = 1800.    # hour in seconds
    day2sec = 86400.  # day in seconds

    # conversion dictionary
    conversion = {'m':1.0, 'mm':0.001, 'cm':0.01, 'km':1000.,
                  'm/s':1.0, 'mm/day':1./(1000.*day2sec), 'kg/m2/s':1./rho_water, 'W/m2':1./latent_heat_evap/rho_water,
                  'kg/kg':1.0, 'g/kg':1.e-3,
                  'fraction':1., "percent":0.01, "%":0.01,
                  'K/s':1., 'K/day':1./day2sec, 
                  'kg/kg/s':1., 'g/kg/hour': rho_water/hr2sec, 
                  'Pa/s':1., 'hPa/day': day2sec/100., 
                  'kg/m2':1., 'g/m2':1.e-3,
                  '1/s':1., '1/hour': 1./hr2sec, 
                  'Pa':1.0, 'hPa':100., "mb":100., 
                 }

    #print(var)
    #if hasattr(var, 'units'):
    #    var.units = unit_out
    #else:
    #    setattr(var,"units",unit_out)
        
    var = var * conversion[unit_in] / conversion[unit_out]
    return var; 

In [ ]:
def get_region_latlon(region):
    """
    ----------------------
    Given a region name, return slices of latitude and longitudes.
    
    Input arguments:
        region = (string) name of the region
            region_list = ["Californian_Sc","Peruvian_Sc","Namibian_Sc","DYCOMS"]

    Return:
        lon_slice & lat_slice of the given region

    Example:
      import yhc_module as yhc
      region = "Californiand_Sc"
      lon_slice, lat_slice = yhc.get_region_latlon(region)
      print(lon_slice)
      print(lat_slice)
      
    Date created: 2022/07/01
    ----------------------
    """

    func_name = "get_region_latlon"
    
    #------------------------
    #  read region name
    #------------------------

    region_list = ["Californian_Sc","Peruvian_Sc","Namibian_Sc","DYCOMS"]
    
    if (region == "Californian_Sc"):
      region_name = "CA marine Sc (20-30N, 120-130W)"
      lowerlat = 20.   # 20N 
      upperlat = 30.   # 30N
      lowerlon = 230.  # 230E
      upperlon = 240.  # 240E

    elif (region == "Peruvian_Sc"): 
      region_name = "Peruvian marine Sc (10S-20S, 80W-90W)"
      lowerlat = -20.  # 20S
      upperlat = -10.  # 10S
      lowerlon = 270.  # 90W
      upperlon = 280.  # 80W

    elif (region == "Namibian_Sc"): 
      region_name = "Namibian marine Sc (10S-20S, 0E-10E)"
      lowerlat = -20.  # 20S
      upperlat = -10.  # 10S
      lowerlon =   0.  # 0E
      upperlon =  10.  # 10E
        
    elif (region == "DYCOMS"): 
      region_name = "DYCOMS (30-32.2N, 120-123.8W)"
      lowerlat =  30.    # 30N
      upperlat =  32.2   # 32.2N
      lowerlon =  236.2  # 123.8W
      upperlon =  240.   # 120W
        
    #elif (region == ""): 
    #  region_name = "(N/S, E/W)"
    #  lowerlat =      # N
    #  upperlat =     # N
    #  lowerlon =    # W
    #  upperlon =  # W
   
    else:
        error_msg = "function *"+func_name+"*: input region ["+region+"] is not supported. STOP. \n" \
                    + "Available regions: "+ ', '.join(region_list)
        sys.exit(error_msg)
    
    #------------------------
    #  compute lon and lat slices
    #    Python slice function: https://www.w3schools.com/PYTHON/ref_func_slice.asp
    #------------------------    

    lon_slice = slice(lowerlon, upperlon)
    lat_slice = slice(lowerlat, upperlat)

    #setattr(lon_slice,"region_long_name","ddd")
    #lon_slice.regionlong_name = "ddd"
    
    return lon_slice, lat_slice

In [ ]:
def wgt_avg (xa,
             dim = ["lon","lat"]
            ):
 
    """
    ----------------------
    Calculate weighted mean of a [*, lon, lat] Xarray data
    
    Input arguments:
      xa : data Array
      dim: dimension to average, e.g. ["lon"], ["lon","lat"]  
      
    Return:
      latitude-weighted average of the input data

    Example
      var is a [*, lon, lat] Xarray data
      
      import yhc_module as yhc
      var_ijmean = yhc.wgt_avg(var)
      var_jmean  = yhc.wgt_avg(var, dim="lat")
        
    References
    1. https://docs.xarray.dev/en/stable/examples/area_weighted_temperature.html
    2. https://nordicesmhub.github.io/NEGI-Abisko-2019/training/Example_model_global_arctic_average.html
 
    Date created: 2022/07/01
    ----------------------
    """
    
    #--- compute latitudal weights
    weights = np.cos(np.deg2rad(xa.lat))
    weights.name = "weights"
    #print(weights)
    
    #--- compute weighted mean
    xa_weighted = xa.weighted(weights)
    xa_weighted_mean = xa_weighted.mean(dim)
    
    return xa_weighted_mean

In [ ]:
def mlevs_to_plevs (ps,
                    model, 
                    plevs, 
                    #model = "AM4_L33_native", 
                    #plevs = "pfull"
                   ):
    """
    ----------------------
    Description: 
        Compute pressure levels from a climate or weather model, given necessary information

    Input arguments:
        ps   : (an xarray DataArray). Surface pressure (Pa)
        model: (a string) model name. Check out "model_list" variable in this function to see which model is supported
        plevs: (a string) return pressure levels. Check out "plevs_list" to see which are supported

        Currently avaialbe:
            model = "AM4_L33_native", GFDL AM4 with 33 levles 
                data.ps MUST be present, surface pressure in Pa
                plevs = ["pfull","phalf"], pressure at full levels or half levels

    Return:
        Input data plus a new dimention that contains pressure levels

    Example:
        import yhc_module as yhc
        data = xr.open_dataset(ncfile)
        data.ps # (time, lat, lon)
        phalf = mlevs_to_plevs(data, plevs="phalf", model = "AM4_L33_native")
        phalf # (time, lat, lon, plev)
     
    Date created: 2022/07/07
    ----------------------
    """

    func_name = "mlevs_to_plevs"
    
    model_list = ["AM4_L33_native"]

#------------------------------------------
# check if the model is supported
#------------------------------------------

    if model not in model_list:
        error_msg = "function *"+func_name+"*: model ["+model+"] is not supported. STOP. \n" \
                    + "Available options: "+ ', '.join(model_list)
        sys.exit(error_msg)
    
#------------------------------------------
# process model and plevs_out
#------------------------------------------

#@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@
    if (model == "AM4_L33_native"):
        """ 
        In AM4, pressure(k) = pk(k) + bk(k)*ps, where ps is surface pressure (Pa).
        """
    
        plevs_list=["phalf", "pfull"]  # AM4 only supports pressure at half levels (phalf) and at full levels (pfull)
    
        #--- check
        if plevs not in plevs_list:        
            error_msg = "function *"+func_name+"*: input plevs ["+plevs+"] is not supported. STOP. \n" \
                      + "Available options: "+ ', '.join(plevs_list)
            sys.exit(error_msg) 
            
        #--- pk values. Output directly from AM4 files
        pk_list = [100, 400, 818.6021, 1378.886, 2091.795, 2983.641, 4121.79, 5579.222, 
          6907.19, 7735.787, 8197.665, 8377.955, 8331.696, 8094.722, 7690.857, 
          7139.018, 6464.803, 5712.357, 4940.054, 4198.604, 3516.633, 2905.199, 
          2366.737, 1899.195, 1497.781, 1156.253, 867.792, 625.5933, 426.2132, 
          264.7661, 145.0665, 60, 15, 0]
 
        #--- bk values. Output directly from AM4 files
        bk_list = [0, 0, 0, 0, 0, 0, 0, 0, 0.00513, 0.01969, 0.04299, 0.07477, 0.11508, 
          0.16408, 0.22198, 0.28865, 0.36281, 0.44112, 0.51882, 0.59185, 0.6581, 
          0.71694, 0.76843, 0.81293, 0.851, 0.88331, 0.91055, 0.93331, 0.95214, 
          0.9675, 0.97968, 0.98908, 0.99575, 1]      

        #--- make pk and bk as xarray.DataArray
        pk = xr.DataArray(pk_list, dims=['plev'])
        bk = xr.DataArray(bk_list, dims=['plev'])    
    
        #--- compute presure at half levels
        phalf = ps*bk + pk
    
        #--- compute pressure at full levels
        pfull = phalf.rolling(plev=2, center=True).mean().dropna("plev")
    
        print(plevs)
        #--- return plevs
        if (plevs == "phalf"):
            return phalf
        elif (plevs == "pfull"):
            return pfull
        
#@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@@@@@@@@@@@@@@@@@@@@@
#elif (model == "other_model"):
#    """ 
#    """        
#    return 
    